In [1]:
import numpy as np
import pandas as pd

In [2]:
molecular_data = pd.read_csv("../../data/preprocessed/rna_df.csv", index_col=0)
drugs = pd.read_csv("../../data/preprocessed/node2vec_embedding.csv", index_col=0)
print(molecular_data.shape, drugs.shape)

(1479, 19193) (500, 1149)


In [3]:
# Load secondary response dataset; Screen is the drug screen across different cell lines
secondary_response_df = pd.read_csv(
    "../../data/raw/prism_19q4_repurpose_primaryfiles/secondary-screen-dose-response-curve-parameters.csv"
)
secondary_response_df.head()

/var/folders/zn/9xk5r70133x08k46ntdqrtcw0000gn/T/ipykernel_34271/137034554.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  secondary_response_df = pd.read_csv('../../data/raw/prism_19q4_repurpose_primaryfiles/secondary-screen-dose-response-curve-parameters.csv')


,broad_id,depmap_id,ccle_name,screen_id,upper_limit,lower_limit,slope,r2,auc,ec50,ic50,name,moa,target,disease.area,indication,smiles,phase,passed_str_profiling,row_name
0,BRD-K71847383-001-12-5,ACH-000879,MFE296_ENDOMETRIUM,HTS002,1,2.122352,-0.022826,-0.026964,1.677789,8.415093e+06,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000879
1,BRD-K71847383-001-12-5,ACH-000320,PSN1_PANCREAS,HTS002,1,1.325174,-0.237504,-0.147274,1.240300,9.643742e+00,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000320
2,BRD-K71847383-001-12-5,ACH-001145,OC316_OVARY,HTS002,1,2.089350,-0.302937,0.193893,1.472333,2.776687e-02,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-001145
3,BRD-K71847383-001-12-5,ACH-000873,KYSE270_OESOPHAGUS,HTS002,1,1.311820,-0.209393,-0.005460,1.207160,2.654701e+00,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000873
4,BRD-K71847383-001-12-5,ACH-000855,KYSE150_OESOPHAGUS,HTS002,1,1.369799,-0.277530,0.132818,1.229332,5.889041e-01,NaN,cytarabine,ribonucleotide reductase inhibitor,"POLA1, POLB, POLD1, POLE",hematologic malignancy,"acute lymphoblastic leukemia (ALL), chronic ly...",Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)...,Launched,True,ACH-000855


In [4]:
auc_labels = secondary_response_df[["depmap_id", "name", "auc"]]
auc_labels = auc_labels.copy()
auc_labels.rename(columns={"depmap_id": "cell_line", "name": "DRUG"}, inplace=True)
auc_labels["DRUG"] = auc_labels["DRUG"].str.upper()

# IC50_labels = IC50_labels.merge(labels_filtered, how = 'inner')
# auc_labels = auc_labels.merge(labels_filtered, how = 'inner')

# Data Cleaning Steps
auc_labels = auc_labels[["cell_line", "DRUG", "auc"]].drop_duplicates()
auc_labels = auc_labels[~np.isnan(auc_labels["auc"])].reset_index(drop=True)
auc_labels = auc_labels[~np.isinf(auc_labels["auc"])].reset_index(drop=True)

# Only keep cell lines in secondary screening where we have data for
auc_labels = auc_labels[np.isin(auc_labels["cell_line"], molecular_data.index)]
auc_labels = auc_labels[np.isin(auc_labels["DRUG"], drugs.columns)]

# Calculate additional statistics
auc_labels["logauc"] = np.log(auc_labels["auc"])
auc_labels["auc_per_drug"] = auc_labels.groupby("DRUG")["logauc"].transform(lambda x: (x - x.mean()) / x.std())
auc_labels["means"] = auc_labels.groupby("DRUG")["logauc"].transform(lambda x: x.mean())
auc_labels["stds"] = auc_labels.groupby("DRUG")["logauc"].transform(lambda x: x.std())
auc_labels["range"] = auc_labels.groupby("DRUG")["logauc"].transform(lambda x: x.max() - x.min())

auc_labels.to_csv("../../data/preprocessed/auc_secondary_screen_prediction_targets.csv")
auc_labels

,cell_line,DRUG,auc,logauc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,0.517477,2.367402,0.228684,0.121987,0.782691
1,ACH-000320,CYTARABINE,1.240300,0.215353,-0.109281,0.228684,0.121987,0.782691
2,ACH-001145,CYTARABINE,1.472333,0.386848,1.296560,0.228684,0.121987,0.782691
3,ACH-000873,CYTARABINE,1.207160,0.188270,-0.331294,0.228684,0.121987,0.782691
4,ACH-000855,CYTARABINE,1.229332,0.206471,-0.182097,0.228684,0.121987,0.782691
...,...,...,...,...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,-0.012671,0.979322,-0.184222,0.175173,1.078361
700917,ACH-001321,AZD8931,1.000000,0.000000,0.879937,-0.199376,0.226580,2.297688
700918,ACH-001321,AZD2014,1.000000,0.000000,1.643680,-0.279217,0.169873,1.025072
700919,ACH-001321,MOTESANIB,0.888088,-0.118685,-0.367226,-0.084827,0.092199,0.957449


In [5]:
# Duplicate check

check_duplicates = auc_labels[["cell_line", "DRUG"]]
# check_duplicates.drop_duplicates()

mask = auc_labels[["cell_line", "DRUG"]]

filtered_df = mask[mask["cell_line"] == "ACH-000879"]
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", None)
print("Rows where 'cell_line' feature contains 'ACH-000879':")
print(filtered_df[filtered_df.duplicated(keep=False)])  # keep=False marks all duplicates as True in the mask

Rows where 'cell_line' feature contains 'ACH-000879':
         cell_line                DRUG
1759    ACH-000879           ADAPALENE
5926    ACH-000879          BELINOSTAT
18608   ACH-000879          BORTEZOMIB
19522   ACH-000879        ESTRAMUSTINE
20893   ACH-000879          DISULFIRAM
22240   ACH-000879        PRALATREXATE
37303   ACH-000879           ERLOTINIB
37771   ACH-000879        TAMIBAROTENE
39046   ACH-000879         REGORAFENIB
47328   ACH-000879           TEPOXALIN
52524   ACH-000879           DOCETAXEL
54341   ACH-000879            OLAPARIB
64131   ACH-000879        CAPECITABINE
66332   ACH-000879          PACLITAXEL
68165   ACH-000879           IMIQUIMOD
69009   ACH-000879   AMINOGLUTETHIMIDE
69911   ACH-000879          MOZAVAPTAN
73229   ACH-000879            GDC-0068
73690   ACH-000879       GAMBOGIC-ACID
75104   ACH-000879          BMS-690514
75569   ACH-000879           FORETINIB
82751   ACH-000879             P276-00
84633   ACH-000879          TOSEDOSTAT
86922   AC